# **Waze Churn Project - hypothesis testing**

**Goal:** Conduct a statistical analysis of ride data based on device type on the provided dataset. Determine, in particular, if there is a statistically significant difference in mean amount of rides between iPhone® users and Android™ users. To do that, apply descriptive statistics and conduct a two-sample hypothesis test (t-test) to analyze the difference in the mean amount of rides between iPhone users and Android users.

*"Do drivers who open the application using an iPhone have the same number of drives on average as drivers who use Android devices?"*

___

## **Imports and data loading**

In [1]:
# Import any relevant packages or libraries
import pandas as pd
from scipy import stats

In [2]:
# Load dataset into dataframe
df = pd.read_csv('data/waze_dataset.csv')

## **Data exploration**

In general, descriptive statistics are useful because they let us quickly explore and understand large amounts of data. In this case, computing descriptive statistics helps us quickly compare the average amount of drives by device type.

In [3]:
df.sample(5)

,ID,label,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device
1619,1619,retained,137,110,181.486438,1045,0,54,4389.512578,3583.366308,22,16,Android
6361,6361,retained,71,58,151.173131,2805,46,0,1349.915163,1000.557768,3,0,Android
4914,4914,retained,134,108,144.296051,2885,531,0,5308.676968,4373.183713,7,6,iPhone
6675,6675,retained,91,74,104.519823,3425,456,0,1732.545511,984.220770,11,4,iPhone
11116,11116,retained,4,4,50.303135,2229,0,46,5051.283121,3717.080029,11,6,iPhone


In [4]:
# summarize data
df.describe(include='all')

,ID,label,sessions,drives,total_sessions,n_days_after_onboarding,total_navigations_fav1,total_navigations_fav2,driven_km_drives,duration_minutes_drives,activity_days,driving_days,device
count,14999.000000,14299,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999
unique,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,retained,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,iPhone
freq,NaN,11763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9672
mean,7499.000000,NaN,80.633776,67.281152,189.964447,1749.837789,121.605974,29.672512,4039.340921,1860.976012,15.537102,12.179879,NaN
std,4329.982679,NaN,80.699065,65.913872,136.405128,1008.513876,148.121544,45.394651,2502.149334,1446.702288,9.004655,7.824036,NaN
min,0.000000,NaN,0.000000,0.000000,0.220211,4.000000,0.000000,0.000000,60.441250,18.282082,0.000000,0.000000,NaN
25%,3749.500000,NaN,23.000000,20.000000,90.661156,878.000000,9.000000,0.000000,2212.600607,835.996260,8.000000,5.000000,NaN
50%,7499.000000,NaN,56.000000,48.000000,159.568115,1741.000000,71.000000,9.000000,3493.858085,1478.249859,16.000000,12.000000,NaN
75%,11248.500000,NaN,112.000000,93.000000,254.192341,2623.500000,178.000000,43.000000,5289.861262,2464.362632,23.000000,19.000000,NaN


In the dataset, `device` is a categorical variable with the labels `iPhone` and `Android`.


In [5]:
## encode device labels into integer

# create a map dictionary
map_dictionary = {'Android': 2, 'iPhone': 1}

# create new `device_type` column
df['device_type'] = df['device']

# map the new column to the dictionary
df['device_type'] = df['device_type'].map(map_dictionary)

df['device_type'].head()

0    2
1    1
2    2
3    1
4    2
Name: device_type, dtype: int64

**Look at the average number of drives for each device type**

In [6]:
# average  number of drives for device_type
df.groupby('device_type')['drives'].mean()

device_type
1    67.859078
2    66.231838
Name: drives, dtype: float64

>Based on the averages shown, it appears that drivers who use an iPhone device to interact with the application have a higher number of drives on average. However, this difference might arise from random sampling, rather than being a true difference in the number of drives.

## **Hypothesis testing**

Recall of the steps for conducting a hypothesis test:
1.   State the null hypothesis and the alternative hypothesis
2.   Choose a signficance level
3.   Find the p-value
4.   Reject or fail to reject the null hypothesis

##### **Perform a two-sample t-test:**

>This is a t-test for two independent samples. This is the appropriate test since the two groups are independent (Android vs. iPhone).


**Hypotheses:**

$H_0$: There is no difference in average number of drives between drivers who use iPhone devices and drivers who use Androids.

$H_A$: There is a difference in average number of drives between drivers who use iPhone devices and drivers who use Androids.

Next, a **significance level of 5%** is choosen.

**Use the `stats.ttest_ind()` function to perform the test.**

In [7]:
# isolate the `drives` column for iPhone users.
iPhone = df[df['device_type'] == 1]['drives']

# isolate the `drives` column for Android users.
Android = df[df['device_type'] == 2]['drives']

# perform the t-test
stats.ttest_ind(a=iPhone, b=Android, equal_var=False)

TtestResult(statistic=1.463523206885235, pvalue=0.143351972680206, df=11345.066049381952)

> *Since the p-value is larger than the chosen significance level (5%), we fail to reject the null hypothesis, and conclude that there is **not** a statistically significant difference in the average number of drives between drivers who use iPhones and drivers who use Androids.*

## **Communicate insights**

**What business insight(s) can be draw from the result of the hypothesis test?**

> *The key business insight is that drivers who use iPhone devices on average have a similar number of drives as those who use Androids.*

> *One potential next step is to explore what other factors influence the variation in the number of drives, and run additonal hypothesis tests to learn more about user behavior. Further, temporary changes in marketing or user interface for the Waze app may provide more data to investigate churn.*

___
___